In [1]:
import cv2
import json
import math
import os
import os.path
import re
import tempfile
from matplotlib import pyplot
from model import airsim as airsim_model
from util import quickstart as util

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Parameters Setting

In [2]:
# 欲載入 pretrain weight 的路徑
MODEL_WEIGHT_PATH = '/data/mjchiu/drone/depth_prediction/model/airsim_2019-12-11-19-21_epoch-100_loss-0.0000.h5'

# 資料路徑
DATA_PATH = '/data/mjchiu/drone/depth_prediction/data/2019-11-20-15-32/scene'
RESULT_PATH = '/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32'

# Model Input 解析度設定
INPUT_HEIGHT = 320
INPUT_WIDTH = 384

# Video Output 解析度設定
OUTPUT_HEIGHT = 1440
OUTPUT_WIDTH = 1920

## Build Model

In [3]:
# 建立 model layer 架構
model = airsim_model.MNv2_segment_depth_multiloss_model(
    inputShape = (INPUT_HEIGHT, INPUT_WIDTH, 3), 
    alpha = 0.35, 
    expansion_factor = 6, 
    depth_multiplier = 1, 
    lock_backend_weights = False,
    CLASSES = 6,
)

# 載入 model weight
model.load_weights(MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet_v2.py:315: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


## Predict the Image and Save Result

In [ ]:
# iterate each dataset
for folder_name in os.listdir(DATA_PATH):
    # prepare result image folder
    result_image_folder_path = os.path.join(RESULT_PATH, 'image', folder_name)
    if not os.path.exists(result_image_folder_path):
        os.makedirs(result_image_folder_path)
    # iterate each image
    folder_path = os.path.join(DATA_PATH, folder_name) 
    for file_name in os.listdir(folder_path):
        # 讀取影像
        scene = util.read_image(os.path.join(folder_path, file_name), shape=(INPUT_HEIGHT,INPUT_WIDTH), rand=0)
        segment = util.read_image(
            os.path.join(folder_path.replace('scene', 'segment'), file_name), 
            shape = (INPUT_HEIGHT,INPUT_WIDTH), 
        )
        # 使用 image 來預測出 depth map 和 semantic segmentation
        predict_depth, predict_segment = util.getResult(model, scene, (INPUT_HEIGHT, INPUT_WIDTH))
        # 儲存結果
        image_path = os.path.join(RESULT_PATH, 'image', folder_name, file_name)
        util.saveResult([scene, segment, predict_depth[0], predict_segment[0]], image_path, (OUTPUT_WIDTH/300, OUTPUT_HEIGHT/300))
    print('predicted dataset {}'.format(folder_name))
            

predicted dataset 2019-11-20-15-45-51
predicted dataset 2019-11-20-15-49-10
predicted dataset 2019-11-20-16-26-03
predicted dataset 2019-11-20-16-06-33
predicted dataset 2019-11-20-16-16-15


# Convert Images into Video 

In [6]:
# prepare result video folder
result_video_folder_path = os.path.join(RESULT_PATH, 'video')
if not os.path.exists(result_video_folder_path):
    os.makedirs(result_video_folder_path)
# iterate result images
result_image_folder =  os.path.join(RESULT_PATH, 'image')
for folder_name in os.listdir(result_image_folder):
    # create result video
    video_path = os.path.join(result_video_folder_path, folder_name+'.mp4')
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'MP4V'), 10, (OUTPUT_WIDTH, OUTPUT_HEIGHT))
    # iterate each image
    for file_name in sorted(os.listdir(os.path.join(result_image_folder, folder_name)), key=lambda x: int(re.split('\\-|\\.', x)[-2])):
        # 儲存結果
        image_path = os.path.join(result_image_folder, folder_name, file_name)
        image = cv2.resize(cv2.imread(image_path), (OUTPUT_WIDTH, OUTPUT_HEIGHT))
        video.write(image)
    video.release()
    print('{} created'.format(video_path))

/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-15-45-51.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-15-49-10.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-16-26-03.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-16-06-33.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-16-16-15.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-15-45-18.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-16-29-39.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-15-52-14.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-15-42-53.mp4 created
/data/mjchiu/drone/depth_prediction/result/2019-11-20-15-32/video/2019-11-20-15-40-54.mp4 created
/data/mjchiu/drone/d